In [3]:
import xarray as xr
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator
import matplotlib.colors as colors
import matplotlib.dates as mdates
from matplotlib.colors import LogNorm
from matplotlib.colors import Normalize
from matplotlib.ticker import FuncFormatter
jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')
import netCDF4 as nc
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter 
import matplotlib.gridspec as gridspec
import os 
import glob
import re
import nc_time_axis
import time 
import dask.array as da
e = np.e
print(e)

2.718281828459045


/tmp/ipykernel_208469/3489755539.py:13: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')


In [8]:
ref_path = '/resstore/b0243/Data/Sporadic_E/FX2100f19f19mg16NaFeMgiontransport_OUTPUT/hist/FX2100f19f19mg16NaFeMgiontransport.cam.h0.2100-01.nc'
ref_file = xr.open_dataset(ref_path, decode_times=False)
ref_file

<xarray.Dataset>
Dimensions:          (lat: 96, lon: 144, mlat: 97, mlon: 80, zlon: 1, nbnd: 2,
                      lev: 126, ilev: 127, time: 1)
Coordinates:
  * lat              (lat) float64 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
  * lon              (lon) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * mlat             (mlat) float64 -90.0 -88.12 -86.24 ... 86.24 88.12 90.0
  * mlon             (mlon) float64 -180.0 -175.5 -171.0 ... 166.5 171.0 175.5
  * zlon             (zlon) float64 0.0
  * lev              (lev) float64 4.055e-10 5.207e-10 6.686e-10 ... 970.6 992.6
  * ilev             (ilev) float64 3.551e-10 4.559e-10 ... 985.1 1e+03
  * time             (time) float64 31.0
Dimensions without coordinates: nbnd
Data variables: (12/587)
    gw               (lat) float64 ...
    zlon_bnds        (zlon, nbnd) float64 ...
    hyam             (lev) float64 ...
    hybm             (lev) float64 ...
    P0               float64 ...
    hyai             (ilev) float64 ...
    ...               ...
    WD_HOCL          (time, lat, lon) float32 ...
    WI               (time, lev, lat, lon) float32 ...
    WTHzm            (time, ilev, lat, zlon) float32 ...
    Z3               (time, lev, lat, lon) float32 ...
    e                (time, lev, lat, lon) float32 ...
    e_SRF            (time, lat, lon) float32 ...
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              FX2100f19f19mg16NaFeMgiontransport
    logname:           earfw
    host:              d8s5b4.arc4.leed
    initial_file:      /resstore/b0154/Data/earfw/MCHEM24/Joe_WACCMX_2_WF_lev...
    topography_file:   /nobackup/earfw/cesm2_inputdata//atm/cam/topo/USGS-gto...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  month_1

In [9]:
ref_file = ref_file.sel(lev=0.0002, time=31.1, method='nearest')

o2_lat = ref_file['O2'].mean(dim='lat')
o2 = o2_lat.mean(dim='lon').item()

In [ ]:
# n2_lat = ref_file['N2'].mean(dim='lat')
# n2 = n2_lat.mean(dim='lon').item()

In [16]:
e_lat = ref_file['EDens'].mean(dim='lat')
e = e_lat.mean(dim='lon').item()

In [17]:
o_lat = ref_file['O'].mean(dim='lat')
o = o_lat.mean(dim='lon').item()

In [ ]:
m = [M, i.e. total atmosphere]

In [19]:
T_lat = ref_file['T'].mean(dim='lat')
T = T_lat.mean(dim='lon').item()
print(T)

188.80032348632812


In [ ]:
k11 = 7.6e-10* (e**(-241/T))          # 'Fe+ + O3 --> FeO+ + O2, from Rollason and Plane
k12 = 3.0e-11                         #'FeO+ + O --> Fe+ + O2
k13 = 4.1e-30*((T/300)**(-1.52*m))    #'Fe+ + N2 + He, from Rollason and Plane
k14 = 8.3e-30*((T/300)**(-1.86*m))    #'Fe+ + O2 + He, from Rollason and Plane
k15 = 5.0e-11                         #'Fe+.N2 or FeO2+ + O --> FeO+ + N2 or O2
k16 = 3.0e-7*((200/T)**0.5)           #'Fe.X+ + e --> Fe + X

In [ ]:
rate = ( (k13*n2+k14*o2)*k15*o/(k15*o+k16*e) + k11*o3)*k16*e/(k16*e+k12*o)
rate = rate + (k13*n2+k14*o2)*(k16*e/(k15*o+k16*e))  

The e-folding lifetime of Fe+ against neutralization is then 1/rate

In [15]:
ref2_path = '/resstore/b0243/Data/Sporadic_E/Solar_Med/archive_yr2000/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h1.0001-01-01-00000.nc'
ref2_file = xr.open_dataset(ref2_path, decode_times=False)
ref2_file

<xarray.Dataset>
Dimensions:       (lat: 96, lon: 144, mlat: 97, mlon: 80, lev: 126, ilev: 127,
                   time: 30, nbnd: 2)
Coordinates:
  * lat           (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon           (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * mlat          (mlat) float64 -90.0 -88.12 -86.24 -84.33 ... 86.24 88.12 90.0
  * mlon          (mlon) float64 -180.0 -175.5 -171.0 ... 166.5 171.0 175.5
  * lev           (lev) float64 4.055e-10 5.207e-10 6.686e-10 ... 970.6 992.6
  * ilev          (ilev) float64 3.551e-10 4.559e-10 5.854e-10 ... 985.1 1e+03
  * time          (time) float64 0.0 1.0 2.0 3.0 4.0 ... 26.0 27.0 28.0 29.0
Dimensions without coordinates: nbnd
Data variables: (12/110)
    gw            (lat) float64 ...
    hyam          (lev) float64 ...
    hybm          (lev) float64 ...
    P0            float64 ...
    hyai          (ilev) float64 ...
    hybi          (ilev) float64 ...
    ...            ...
    UI            (time, lev, lat, lon) float32 ...
    V             (time, lev, lat, lon) float32 ...
    VI            (time, lev, lat, lon) float32 ...
    WI            (time, lev, lat, lon) float32 ...
    Z3            (time, lev, lat, lon) float32 ...
    e             (time, lev, lat, lon) float32 ...
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport
    logname:           earfw
    host:              d13s7b4.arc4.lee
    initial_file:      /nobackup/earfw/cesm2/cases/ACP_CESM213_FX2000_f19_f19...
    topography_file:   /nobackup/earfw/cesm2_inputdata//atm/cam/topo/USGS-gto...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  day_1